In [1]:
import os

# Set up input and output paths
input_dir1 = r"Q:\@STENGL\SVP\MAPY\-- PROJEKT - MPO-MPR --\rastre\UST_HLBKY"
input_dir2 = r"Q:\@STENGL\SVP\MAPY\-- PROJEKT - MPO-MPR --\rastre\UST_RYCHLOSTI"
output_gdb = r"D:\RASTRE\RASTRE_TEMP.gdb"
# Set the input and output paths for CO
input_path = r"D:\RASTRE\RASTRE_TEMP.gdb"
output_path = r"D:\RASTRE\RASTRE_TEMP.gdb"
arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "100%"

In [2]:
# Set the subfolder names 
subfolders = ["HN_02a_HDM"]

In [3]:
# VÝPOČET INTEZITY POVODNE - IP
for folder in subfolders:
    # Set the input folder paths
    input_path1 = os.path.join(input_dir1, folder)
    input_path2 = os.path.join(input_dir2, folder)
    
    # Identify the input rasters to use for the calculation q10,q100
    q10_raster1 = os.path.join(input_path1, f"{folder}_hlbka_q0010.tif") #0010
    q10_raster2 = os.path.join(input_path2, f"{folder}_rychlost_q0010.tif") #0010
    q100_raster1 = os.path.join(input_path1, f"{folder}_hlbka_q0100.tif") #0100
    q100_raster2 = os.path.join(input_path2, f"{folder}_rychlost_q0100.tif") #0100
    
    #q1000
    q1000_raster3 = os.path.join(input_path1, f"{folder}_hlbka_q1000.tif")
    
    # Create raster objects q10/100
    q10_raster1_obj = arcpy.Raster(q10_raster1)
    q10_raster2_obj = arcpy.Raster(q10_raster2)
    q100_raster1_obj = arcpy.Raster(q100_raster1)
    q100_raster2_obj = arcpy.Raster(q100_raster2)
    
    #q1000
    q100_raster3_obj = arcpy.Raster(q1000_raster3)
    
    # Calculation variabels from path 
    output_q10 = os.path.join(output_gdb, f"{folder}_IP_q10")
    output_q100 = os.path.join(output_gdb, f"{folder}_IP_q100")
    output_q1000 = os.path.join(output_gdb, f"{folder}_IP_q1000_r") 
    
    # Perform the raster calculation for q10
    q10_cond = arcpy.sa.Con(q10_raster1_obj > 0, arcpy.sa.Con(q10_raster2_obj <= 1, q10_raster1_obj, q10_raster1_obj * q10_raster2_obj))
    q10_cond.save(output_q10)
    print(f"Raster {folder}_IP_Q10 done.")
    # Perform the raster calculation for q100
    q100_cond = arcpy.sa.Con(q100_raster1_obj > 0, arcpy.sa.Con(q100_raster2_obj <= 1, q100_raster1_obj, q100_raster1_obj * q100_raster2_obj))
    q100_cond.save(output_q100)
    print(f"Raster {folder}_IP_Q100 done.")
    # Perform the raster calculation for q1000
    q1000_cond = arcpy.sa.Con(q100_raster3_obj > 0, 1)
    q1000_cond.save(output_q1000)
    print(f"Raster {folder}_IP_Q1000_r done.")

Raster HN_02a_HDM_IP_Q10 done.
Raster HN_02a_HDM_IP_Q100 done.
Raster HN_02a_HDM_IP_Q1000_r done.


In [4]:
# VÝPOČET INTEZITY POVODNE - IP_Q_r
# Set the input paths for the new calculation
ip_q10_r = arcpy.Raster(os.path.join(output_gdb, f"{folder}_IP_q10"))
ip_q100_r = arcpy.Raster(os.path.join(output_gdb, f"{folder}_IP_q100"))

In [5]:
# Perform the raster calculation for the new output on q10_r
output_q10_r = (0.3 + 1.35 * ip_q10_r) * 0.1
output_q10_r.save(os.path.join(output_gdb, f"{folder}_q10_r"))
print(f"Raster {folder}_IP_Q10_r done.")
# Perform the raster calculation for the new output on q100_r
output_q100_r = (0.3 + 1.35 * ip_q100_r) * 0.01
print(f"Raster {folder}_IP_Q100_r done.")
output_q100_r.save(os.path.join(output_gdb, f"{folder}_q100_r"))

Raster HN_02a_HDM_IP_Q10_r done.
Raster HN_02a_HDM_IP_Q100_r done.


In [6]:
# Apply symbology for Q10
arcpy.management.ApplySymbologyFromLayer(
    in_layer=f"{input_path}\\HN_02a_HDM_q10_r",
    in_symbology_layer=r"TEMP\\BG_18_IP_q10_r",
    symbology_fields=None,
    update_symbology="MAINTAIN"
)
# Apply symbology for Q100
arcpy.management.ApplySymbologyFromLayer(
    in_layer=f"{input_path}\\HN_02a_HDM_q100_r",
    in_symbology_layer=r"TEMP\\BG_18_IP_q100_r",
    symbology_fields=None,
    update_symbology="MAINTAIN"
)

# Reclassify Q10
out_raster_q10 = f"{output_path}\\Reclass_HN_02a_Q10"
arcpy.sa.Reclassify(
    in_raster=f"{input_path}\\HN_02a_HDM_q10_r",
    reclass_field="VALUE",
    remap="0,030002 0,099000 3;0,099000 10 4",
    missing_values="DATA"
).save(out_raster_q10)
# Reclassify Q100
out_raster_q100 = f"{output_path}\\Reclass_HN_02a_Q100"
arcpy.sa.Reclassify(
    in_raster=f"{input_path}\\HN_02a_HDM_q100_r",
    reclass_field="VALUE",
    remap="0,003000 0,010000 2;0,010000 0,029900 3;0,029900 10 4",
    missing_values="DATA"
).save(out_raster_q100)

# Reclassify Q10
out_raster_q10 = f"{output_path}\\Reclass_HN_02a_Q10"
arcpy.sa.Reclassify(
    in_raster=f"{input_path}\\HN_02a_HDM_q10_r",
    reclass_field="VALUE",
    remap="0,030002 0,099000 3;0,099000 10 4",
    missing_values="DATA"
).save(out_raster_q10)
# Reclassify Q100
out_raster_q100 = f"{output_path}\\Reclass_HN_02a_Q100"
arcpy.sa.Reclassify(
    in_raster=f"{input_path}\\HN_02a_HDM_q100_r",
    reclass_field="VALUE",
    remap="0,003000 0,010000 2;0,010000 0,029900 3;0,029900 10 4",
    missing_values="DATA"
).save(out_raster_q100)

with arcpy.EnvManager(extent="MAXOF"):
    # Define the input rasters and their corresponding names
    rasters = [out_raster_q10, out_raster_q100, f"{input_path}\\HN_02a_HDM_IP_q1000_r"]
    input_names = ["out_raster_q10", "out_raster_q100", "HN_02a_HDM_IP_q1000_r"]

    # Use the input rasters and names in the RasterCalculator expression
    expression = 'Con(IsNull(out_raster_q10), Con(IsNull(out_raster_q100), HN_02a_HDM_IP_q1000_r, out_raster_q100), out_raster_q10)'

    output_raster = arcpy.sa.RasterCalculator(
        expression=expression,
        rasters=rasters,
        input_names=input_names
    )

output_raster.save(f"{output_path}\\HN_02a_CO")